#### 수집 데이터 전처리

##### 전처리용 모듈 임포트

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_excel('./2_다나와_TV크롤링결과.xlsx')
df.tail()

,상품명,스펙목록,가격
2395,디엘티 ZEUS 아르떼 D3255H,LED TV / 32인치(81cm) / LED / HD / 60Hz / 명암비: 5...,135600
2396,삼성전자 LH65QHBEBGCXKR,실내용 사이니지 / 디스플레이모듈 / LED / 65인치(164cm) / 4K UH...,1349950
2397,와사비망고 QG551 QLED 와글와글 플러스,QLED TV / 55인치(138cm) / QLED / 4K UHD / VA패널계열...,439000
2398,삼성전자 LH32DCE2LGA,실내용 사이니지 / 디스플레이모듈 / LED / 32인치(80cm) / FHD / ...,439000
2399,삼성전자 UN40N5200 리퍼비시,LED TV / 40인치(101cm) / LED / FHD / 프로세서: 하이퍼리얼...,399000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품명     2400 non-null   object
 1   스펙목록    2400 non-null   object
 2   가격      2400 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 56.4+ KB


In [7]:
# 제조사 상품명 [상품코드]
# split시 두번째 파라미터 숫자가 자를 횟수
df['상품명'][725].split(' ', 1)

['필립스', '55PUN7645']

In [8]:
comp_list, prod_list = [], []
count = 0

for title in df['상품명']:
    temps = str(title).split(' ', 1)
    try:
        if len(temps) == 2: # 회사명/모델명만 있는 상품이 존재
            comp_list.append(temps[0])
            prod_list.append(temps[1])
        else:
            comp_list.append(temps[0])
            prod_list.append(temps[0])
    except:
        print(count)
        
    count += 1

In [9]:
cate_list, usetime_list, suction_list = [], [], []


for spec_data in df['스펙목록']:
    spec_list = spec_data.split(' / ')


    # 카테고리, 사용시간, 흡입력 만 추출
    category = spec_list[0]
    cate_list.append(category)

    time_count = 0
    suction_count = 0 # 사용시간, 흡입력이 있을떄만 카운트

    for spec in spec_list:
        if '사용시간' in spec:
            usetime_value = str(spec).split(' ')[1].strip()
            time_count +=1
        if '소비전력' in spec:
            suction_value = str(spec).split(' ')[1].strip()
            suction_count +=1

    if time_count == 0: # 사용시간정보가 없음
        usetime_value = ''
    if suction_count == 0: # 흡입력에 대한 정보가 없음
        suction_value = ''
    
    usetime_list.append(usetime_value)
    suction_list.append(suction_value)

In [10]:
usetime_list[16]

''

##### 사용시간, 흡입력 단위 통일, 수치화

- 사용시간 : 분, 시간, 시간 분 등 분류가 되어 있고, object64 --> 수치화 할 수 없음
- 흡입력 : W, AW, Pa 등으로 단위가 일치하지 않고, object64 --> 수치화 불가

In [17]:
# 사용시간 단위 분으로 변환 함수

def convert_to_min(cur_time):
    hour, minute = 0, 0

    try:
        if '시간' in cur_time:
            hour = int(cur_time.split('시간')[0])

            if '분' in cur_time:
                minute = int(cur_time.split('시간')[-1].split('분')[0])
            else:
                minute = 0
        else:
            hour = 0
            minute = int(cur_time.split('분')[0])    

        return hour*60 + minute
    
    except:
        return None
    
print(convert_to_min(usetime_list[0]))


180


In [18]:
usemin_list = []

for ctime in usetime_list:
    usemin_list.append(convert_to_min(ctime))


In [19]:
pd.DataFrame(usemin_list)

,0
0,180.0
1,180.0
2,NaN
3,NaN
4,NaN
...,...
2395,NaN
2396,NaN
2397,NaN
2398,NaN


In [260]:
usemin_list[2222]

In [212]:
# 흡입력 단위 통일
def convert_suction(value):
    try:
        value = str(value).upper() # AW, W, Pa -> PA
        if 'AW' in value or 'W' in value:
            result = value.replace('A', '').replace('W', '').replace(',', '')
            result = int(result)
        elif 'PA' in value:
            result = value.replace('PA', '')
            result = int(result)/100
        else:
            result = None
    except:
        result = None
    
    return result

In [225]:
convert_suction(suction_list[19])

220

In [227]:
new_suction_list = []

for power in suction_list:
    new_suction_list.append(convert_suction(power))

In [228]:
len(new_suction_list)

2225

#### 전처리 완료 자료 DF생성

In [229]:
df_product = pd.DataFrame()
df_product['카테고리'] = cate_list
df_product['회사명'] = comp_list
df_product['제품명'] = prod_list
df_product['가격'] = df['가격']
df_product['사용시간'] = usemin_list
df_product['흡입력'] = new_suction_list

In [232]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   카테고리    2225 non-null   object 
 1   회사명     2225 non-null   object 
 2   제품명     2225 non-null   object 
 3   가격      2225 non-null   object 
 4   사용시간    1687 non-null   float64
 5   흡입력     909 non-null    float64
dtypes: float64(2), object(4)
memory usage: 104.4+ KB


In [283]:
# 가격이 object인 이유는 '일시품절' --> 0
df_product[df_product['가격'] == '일시품절'] = None


In [285]:
df_product['카테고리'].value_counts()

카테고리
핸디스틱청소기              1034
물걸레청소기                357
차량용청소기                301
핸디청소기                 169
침구청소기                 107
욕실청소기                  69
업소용청소기                 45
핸디/스틱청소기+로봇청소기세트       41
스틱청소기                  32
0                      32
스틱청소기+로봇청소기트            5
진공청소기                   4
스틱청소기+공기청정기세트           4
스팀청소기                   4
청소기+제습기                 3
핸디/스틱청소기+핸디/스틱청소기       3
물걸레청소기+스팀청소기            2
침구/카펫청소기                2
핸디스틱청소기+공기청정기세트         2
청소기+공기청정기               2
로봇청소기                   1
차량용선풍기                  1
핸디/스틱청소기+전기밥세트          1
핸디/스틱청소기+물걸레청소기세트       1
핸디/스틱청소기+업소용청소기         1
욕실+물걸레청소기               1
수영장청소기                  1
Name: count, dtype: int64

In [286]:
df_product_final = df_product[df_product['카테고리'].isin(['핸디스틱청소기'])]

In [290]:
df_product_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1034 entries, 0 to 1367
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   카테고리    1034 non-null   object 
 1   회사명     1034 non-null   object 
 2   제품명     1034 non-null   object 
 3   가격      1034 non-null   int64  
 4   사용시간    899 non-null    float64
 5   흡입력     674 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 56.5+ KB


In [288]:
df_product_final = df_product_final.astype({'가격':'int64'}) # object64 --> int64 변경

In [289]:
df_product_final.to_excel('./3_다나와_핸디스틱청소기_결과.xlsx', index=False)